In [7]:
import os

In [8]:
%pwd

'D:\\Projects\\SmartBin-Intelligent-Waste-Sorting-with-IoT-and-Machine-Learning'

In [9]:
os.chdir("D:\Projects\SmartBin-Intelligent-Waste-Sorting-with-IoT-and-Machine-Learning")

In [10]:
%pwd

'D:\\Projects\\SmartBin-Intelligent-Waste-Sorting-with-IoT-and-Machine-Learning'

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [12]:
from SmartBin.constants import *
from SmartBin.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [14]:
import os
import urllib.request as request
import zipfile
from SmartBin import logger
from SmartBin.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-25 20:35:13,642: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-25 20:35:13,643: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-25 20:35:13,645: INFO: common: created directory at: artifacts]
[2024-01-25 20:35:13,646: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-25 20:36:07,853: INFO: 4037744573: artifacts/data_ingestion/Recycle_Data.zip download! with following info: 
x-amz-id-2: TYfFPb0oYsRlB8qSJfb00IIUJWP7516P+Q1cn/9ZxKK3KIg+VLbJYTSMZKxeka2C3h6yhRx0JgA=
x-amz-request-id: P830D67KRT90SSK8
Date: Thu, 25 Jan 2024 20:35:12 GMT
Last-Modified: Tue, 23 Jan 2024 00:35:57 GMT
ETag: "1632733a6d5e761b3cd910f3d116b5ff-27"
x-amz-server-side-encryption: AES256
Accept-Ranges: bytes
Content-Type: application/zip
Server: AmazonS3
Content-Length: 447914276
Connection: close

]
